In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
from dotenv import load_dotenv
load_dotenv()
from langchain.embeddings.openai import OpenAIEmbeddings
from pymongo import MongoClient
import os
import pandas as pd
import uuid

# Excel 파일 읽기
xlsx_path = 'test용사전엑셀.xlsx'
df = pd.read_excel(xlsx_path)


# MongoDB 연결
MONGODB_ATLAS_CLUSTER_URI = os.getenv("MONGODB_ATLAS_CLUSTER_URI")
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

DB_NAME = "test"
COLLECTION_NAME = "col"
collection = client[DB_NAME][COLLECTION_NAME]

# 데이터 처리 및 저장
documents = []
for _, row in df.iterrows():
    doc = {
        "_id": str(uuid.uuid4()),
        "metadata": {
            "KO": str(row['ko']),
            "EN": str(row['en']),
            "JAP": str(row['jap'])
        }
    }
    documents.append(doc)

# 기존 데이터 삭제 (선택사항)
collection.delete_many({})


embeddings = OpenAIEmbeddings()


# 벌크 삽입
if documents:
    collection.insert_many(documents)

# 다국어 검색을 위한 인덱스 생성
collection.create_index([
    ("metadata.KO", "text"),
    ("metadata.EN", "text"),
    ("metadata.JAP", "text")
])

# 검색 함수
def search_dictionary(query_text, language="KO"):
    """
    지정된 언어로 사전 검색을 수행합니다.
    
    Parameters:
    - query_text: 검색할 텍스트
    - language: 검색 언어 (KO, EN, JAP)
    """
    query = {f"metadata.{language}": {"$regex": query_text, "$options": "i"}}
    results = collection.find(query)
    return list(results)

# 데이터 확인
print("저장된 첫 번째 문서:")
print(next(collection.find({})))

# 검색 예시
# ko_results = search_dictionary("커피", language="KO")
# print("\n한글 검색 결과:")
# for result in ko_results:
#     print(result)

저장된 첫 번째 문서:
{'_id': '17123f0f-bdd2-4195-8c39-a78c1c3bf282', 'metadata': {'KO': '산독끼', 'EN': 'Zinrabit', 'JAP': '景福宮'}}


In [ ]:
from dotenv import load_dotenv
load_dotenv()
from langchain.embeddings.openai import OpenAIEmbeddings
from pymongo import MongoClient
import os
import pandas as pd
import uuid

# Excel 파일 읽기
xlsx_path = 'test용사전엑셀.xlsx'
df = pd.read_excel(xlsx_path)


# MongoDB 연결
MONGODB_ATLAS_CLUSTER_URI = os.getenv("MONGODB_ATLAS_CLUSTER_URI")
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

DB_NAME = "test"
COLLECTION_NAME = "col"
collection = client[DB_NAME][COLLECTION_NAME]

# 데이터 처리 및 저장
documents = []
for _, row in df.iterrows():
    doc = {
        "_id": str(uuid.uuid4()),
        "metadata": {
            "KO": str(row['ko']),
            "EN": str(row['en']),
            "JAP": str(row['jap'])
        }
    }
    documents.append(doc)

# 기존 데이터 삭제 (선택사항)
collection.delete_many({})


embeddings = OpenAIEmbeddings()


# 벌크 삽입
if documents:
    collection.insert_many(documents)

# 다국어 검색을 위한 인덱스 생성
collection.create_index([
    ("metadata.KO", "text"),
    ("metadata.EN", "text"),
    ("metadata.JAP", "text")
])

# 검색 함수
def search_dictionary(query_text, language="KO"):
    """
    지정된 언어로 사전 검색을 수행합니다.
    
    Parameters:
    - query_text: 검색할 텍스트
    - language: 검색 언어 (KO, EN, JAP)
    """
    query = {f"metadata.{language}": {"$regex": query_text, "$options": "i"}}
    results = collection.find(query)
    return list(results)

# 데이터 확인
print("저장된 첫 번째 문서:")
print(next(collection.find({})))

# 검색 예시
# ko_results = search_dictionary("커피", language="KO")
# print("\n한글 검색 결과:")
# for result in ko_results:
#     print(result)

저장된 첫 번째 문서:
{'_id': '17123f0f-bdd2-4195-8c39-a78c1c3bf282', 'metadata': {'KO': '산독끼', 'EN': 'Zinrabit', 'JAP': '景福宮'}}


In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import os
import re


def find_matching_words(text, language_from="KO", language_to="EN"):
    """
    텍스트에서 단어를 추출하고 MongoDB에서 매칭되는 단어들을 찾습니다.
    """
    # 텍스트에서 단어 추출
    words = re.findall(r'\w+', text)
    
    matched_translations = []
    for word in words:
        # MongoDB에서 단어 검색
        result = collection.find_one({f"metadata.{language_from}": word})
        if result:
            matched_translations.append({
                'original': word,
                'translation': result['metadata'][language_to]
            })
    
    return matched_translations

# 번역을 위한 PromptTemplate 설정
translation_prompt = PromptTemplate(
    input_variables=["dictionary_matches", "query"],
    template="""
다음은 번역에 참고할 수 있는 단어 사전입니다:
{dictionary_matches}

위 단어 사전을 참고하여, 아래 한국어 문장을 영어로 번역해주세요.
문장: {query}
번역:
"""
)

# LLM 체인 설정
trans_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
translation_chain = LLMChain(
    llm=trans_llm,
    prompt=translation_prompt
)

def translate_with_dictionary(query, from_lang="KO", to_lang="EN"):
    """
    MongoDB 사전을 활용하여 텍스트를 번역합니다.
    """
    # 매칭되는 단어들 찾기
    matches = find_matching_words(query, from_lang, to_lang)
    
    # 사전 매칭 결과를 문자열로 변환
    dictionary_matches = "\n".join([
        f"- {match['original']} → {match['translation']}"
        for match in matches
    ])
    
    if not dictionary_matches:
        dictionary_matches = "매칭되는 단어가 사전에 없습니다."
    
    # 번역 실행
    result = translation_chain.run({
        "dictionary_matches": dictionary_matches,
        "query": query
    })
    
    return {
        "translation": result,
        "matched_words": matches
    }



In [19]:
# 사용 예시
query = "산독끼는 바나나와 꼬라니를 좋아해."
result = translate_with_dictionary(query)

print("찾은 단어들:")
for match in result["matched_words"]:
    print(f"- {match['original']} → {match['translation']}")

print("\n최종 번역 결과:", result["translation"])

찾은 단어들:

최종 번역 결과: The mountain rat likes bananas and corn.
